In [2]:
from models.Model import Model
from models.Logistic.MultiGPT2Logistic import MultiGPT2Logistic
from languages.LanguageModel import LanguageModel
from DataExploration import DataExploration
# from languages.Japanese import Japanese
from languages.English import English
from languages.Finnish import Finnish
from Preprocess import Preprocess
from Pipeline import Pipeline
from typing import List
import torch
import datasets
import pandas as pd
import random
import numpy as np

In [3]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently 
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

enforce_reproducibility()

In [4]:
# Is used to minimize the clutter in the console
datasets.logging.set_verbosity_error()

# Define the languages to be used
languages: List[LanguageModel] = [
    English(),
    Finnish(),
    # Japanese()
]

# gpt2Generator = GPT2Generator()
torch.cuda.empty_cache()


# Preprocessing and Dataset Analysis

In [5]:
all_data = {}

for language in languages:
    pipeline = Pipeline()

    # Get the preprocessed data and split it into training and validation data
    preprocessor = Preprocess(language.tokenize, language.clean)
    data = pipeline.get_data(language=language.name, preproccesor=preprocessor)
    train_data, validation_data = pipeline.split_data(data)

    all_data[language.name] = {
        "train_data": train_data,
        "validation_data": validation_data
    }


Loading data...

Loading data...


# Multilingual QA

We start by implementing our binary question system with a multilingual encoder instead of the monolingual ones. With this we perform zero-shot cross-lingual evaluation

In [6]:
# Zero shot classification
for train_language in languages:
    print(f'\n\n--- Training on {train_language.name} ---')
    model = MultiGPT2Logistic()
    model.set_language(train_language.name)
    
    try:
        model.load()
    except:
        train_data = all_data[train_language.name]["train_data"]
        X_train = model.extract_X(train_data)
        y_train = train_data['is_answerable']
        model = pipeline.train(
            model,
            X_train,
            y_train
        )
        model.save()
    
    for val_language in languages:
        print(f'\n\t- Validating on {val_language.name}')
        validation_data = all_data[val_language.name]["validation_data"]
        X_validation = model.extract_X(validation_data)
        y_validation = validation_data['is_answerable']
        pipeline.evaluate(
            model,
            X_validation,
            y_validation
        )



--- Training on english ---


Using eos_token, but it is not set yet.
